In [49]:
from dotenv import load_dotenv
import os
import pandas as pd
import numpy as np
from sqlalchemy import Integer, Float, String, Boolean, DateTime, Interval, Text
from sqlalchemy import create_engine
from sqlalchemy.dialects.postgresql import JSONB
pd.set_option('display.max_columns', None)

### Load evn variables

In [50]:
load_dotenv()

True

### SQL setup

In [51]:
engine = create_engine(os.getenv('DB_URI'))

## Request data

### All activities Dataframe

In [52]:
activitie_list_query = "SELECT * FROM bronze.activities"
activities_list_df = pd.read_sql(activitie_list_query, engine)

### All activities with details Dataframe

In [53]:
activities_details_query = "SELECT * FROM bronze.activities_details"
activities_details_df = pd.read_sql(activities_details_query, engine)

### All kudos Dataframe

In [54]:
kudos_query = "SELECT * FROM bronze.kudos"
kudos_df = pd.read_sql(kudos_query, engine)

### Separate tables setup

In [55]:
dataframe_columns = {
  'activities' : [
    'id',
    'name',
    'distance',
    'moving_time',
    'elapsed_time',
    'total_elevation_gain',
    'type',
    'sport_type',
    'workout_type',
    'start_date',
    'start_date_local',
    'timezone',
    'utc_offset',
    'location_city',
    'location_state',
    'location_country',
    'achievement_count',
    'kudos_count',
    'comment_count',
    'athlete_count',
    'photo_count',
    'trainer',
    'commute',
    'manual',
    'private',
    'visibility',
    'flagged',
    'start_latlng',
    'end_latlng',
    'average_speed',
    'max_speed',
    'average_cadence',
    'average_watts',
    'max_watts',
    'weighted_average_watts',
    'device_watts',
    'kilojoules',
    'has_heartrate',
    'average_heartrate',
    'max_heartrate',
    'heartrate_opt_out',
    'display_hide_heartrate_option',
    'elev_high',
    'elev_low',
    'upload_id',
    'upload_id_str',
    'external_id',
    'from_accepted_tag',
    'pr_count',
    'total_photo_count',
    'has_kudoed',
    'suffer_score',
    'description',
    'calories',
    'perceived_exertion',
    'prefer_perceived_exertion',
    'hide_from_home',
    'device_name',
    'embed_token',
    'available_zones',
    'map_id',
    'gear_id'],
  'maps' : [
    'map_id',
    'map_polyline',
    'map_resource_state',
    'map_summary_polyline'],
  'gear' : [
    'gear_id',
    'gear_primary',
    'gear_name',
    'gear_nickname',
    'gear_resource_state',
    'gear_retired',
    'gear_distance',
    'gear_converted_distance'],
  'segment_efforts' : [
    'id',
    'resource_state',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'start_index',
    'end_index',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'pr_rank',
    'achievements',
    'visibility',
    'kom_rank',
    'hidden',
    'activity_id',
    'segment_id'],
  'segments' : [
    'segment_id',
    'segment_resource_state',
    'segment_name',
    'segment_activity_type',
    'segment_distance',
    'segment_average_grade',
    'segment_maximum_grade',
    'segment_elevation_high',
    'segment_elevation_low',
    'segment_start_latlng',
    'segment_end_latlng',
    'segment_elevation_profile',
    'segment_elevation_profiles',
    'segment_climb_category',
    'segment_city',
    'segment_state',
    'segment_country',
    'segment_private',
    'segment_hazardous',
    'segment_starred'],
  'laps' : [
    'id',
    'resource_state',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'average_speed',
    'max_speed',
    'lap_index',
    'split',
    'start_index',
    'end_index',
    'total_elevation_gain',
    'average_cadence',
    'device_watts',
    'average_watts',
    'average_heartrate',
    'max_heartrate',
    'pace_zone',
    'activity_id'],
  'best_efforts' : [
    'id',
    'activity_id',
    'resource_state',
    'name',
    'elapsed_time',
    'moving_time',
    'start_date',
    'start_date_local',
    'distance',
    'pr_rank',
    'achievements',
    'start_index',
    'end_index']
}

### Spliting data into tables

In [56]:
def select_cols(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
  """
  Selects only the specified columns from a DataFrame if they exist.

  Parameters
  ----------
  df : pd.DataFrame
      The input DataFrame.
  columns : list of str
      List of column names to select.

  Returns
  -------
  pd.DataFrame
      A new DataFrame containing only the specified columns that exist 
      in the input DataFrame. If none of the columns exist, 
      an empty DataFrame is returned.
  """
  return df[[c for c in cols if c in df.columns]].copy()

def explode_normalize_json(df: pd.DataFrame, col: str) -> pd.DataFrame:
  """
  Explodes a list-like column in a DataFrame and normalizes nested JSON records into a flat table.

  Parameters
  ----------
  df : pd.DataFrame
      Input DataFrame containing a column with list-like or dictionary-like structures.
  col : str
      The name of the column to explode and normalize.

  Returns
  -------
  pd.DataFrame
      A new DataFrame with the exploded and normalized JSON records.
      If the column does not exist or contains only empty values, 
      an empty DataFrame is returned.
  """

  if col not in df.columns:
    return pd.DataFrame()
  
  exploded = df.explode(col).reset_index(drop=True)
  exploded_values = exploded[col].dropna()

  if exploded_values.empty:
    return pd.DataFrame()
  
  return pd.json_normalize(exploded_values, sep='_')

In [57]:
# Activities
activities_cols = dataframe_columns['activities']
activities_df = activities_details_df[[c for c in activities_cols if c in activities_details_df.columns]].copy()

# Maps
maps_cols = dataframe_columns['maps']
maps_df = activities_details_df[[c for c in maps_cols if c in activities_details_df.columns]].copy()


# Gear
gear_cols = dataframe_columns['gear']
gear_df = activities_details_df[[c for c in gear_cols if c in activities_details_df.columns]].copy()
gear_df = gear_df.drop_duplicates()

# Segment efforts
seg_eff_cols = dataframe_columns['segment_efforts']
segments_eff_exploded_df = activities_details_df.copy().explode('segment_efforts').reset_index(drop=True)
segments_eff_df = pd.json_normalize(segments_eff_exploded_df['segment_efforts'], sep='_')
segments_eff_df = segments_eff_df[[c for c in seg_eff_cols if c in segments_eff_df.columns]]

# Segments
seg_cols = dataframe_columns['segments']
segments_exploded_df = activities_details_df.copy().explode('segment_efforts').reset_index(drop=True)
segments_df = pd.json_normalize(segments_exploded_df['segment_efforts'], sep='_')
segments_df = segments_df[[c for c in seg_cols if c in segments_df.columns]]

# Laps
lap_cols = dataframe_columns['laps']
laps_exploded_df = activities_details_df.copy().explode('laps').reset_index(drop=True)
laps_df = pd.json_normalize(laps_exploded_df['laps'], sep='_')
laps_df = laps_df[[c for c in lap_cols if c in laps_df.columns]]

# Best efforts
best_eff_cols = dataframe_columns['best_efforts']
best_eff_exploded_df = activities_details_df.copy().explode('best_efforts').reset_index(drop=True)
best_eff_df = pd.json_normalize(best_eff_exploded_df['best_efforts'], sep='_')
best_eff_df = best_eff_df[[c for c in lap_cols if c in best_eff_df.columns]].dropna(how="all")

# All dataframes in dictoinary
dataframes = {
    "activities": activities_df,
    "maps": maps_df,
    "gear": gear_df,
    "segment_efforts": segments_eff_df,
    "segments": segments_df,
    "laps": laps_df,
    "best_efforts": best_eff_df,
    "kudos" : kudos_df
}

In [58]:
workout_types = [
    {"id": 0.0, "type": "Running - None"},
    {"id": 1.0, "type": "Running - Race"},
    {"id": 2.0, "type": "Running - Long Run"},
    {"id": 3.0, "type": "Running - Workout"},
    {"id": 10.0, "type": "Riding - None"},
    {"id": 11.0, "type": "Riding - Race"},
    {"id": 12.0, "type": "Riding - Race"},
    {"id": 20.0, "type": "Other"}
]

In [59]:
def speed_to_pace_str(speed: float) -> str | None:
  """
  Converts speed in meters per second to running pace in the format "M:SS per km".

  Parameters
  ----------
  speed : float
      Speed value in meters per second. Must be greater than zero.

  Returns
  -------
  str or None
      A string representing the pace in minutes and seconds per kilometer 
      (e.g., "5:32"). Returns None if the speed is less than or equal to zero.
  """

  if speed <= 0:
    return None
  
  seconds = 1000/speed
  minutes = int(seconds // 60)
  sec = int(round(seconds % 60))

  if sec == 60:
    minutes += 1
    sec = 0

  return f"{minutes}:{sec:02d}"

def speed_to_pace_float(speed: float) -> float | None:

  """
  Converts speed in meters per second to running pace in minutes per kilometer (float).

  Parameters
  ----------
  speed : float
      Speed value in meters per second. Must be greater than zero.

  Returns
  -------
  float or None
      Running pace in minutes per kilometer, represented as a float 
      (e.g., 5.53 means 5.53 minutes per km). 
      Returns None if the speed is less than or equal to zero.
  """

  if speed <= 0:
    return None
  
  seconds = 1000/speed

  return seconds / 60

In [60]:
activities_df

,id,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,start_date,start_date_local,timezone,utc_offset,location_city,location_state,location_country,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,private,visibility,flagged,start_latlng,end_latlng,average_speed,max_speed,average_cadence,average_watts,max_watts,weighted_average_watts,device_watts,kilojoules,has_heartrate,average_heartrate,max_heartrate,heartrate_opt_out,display_hide_heartrate_option,elev_high,elev_low,upload_id,upload_id_str,external_id,from_accepted_tag,pr_count,total_photo_count,has_kudoed,suffer_score,description,calories,perceived_exertion,prefer_perceived_exertion,hide_from_home,device_name,embed_token,available_zones,map_id,gear_id
0,15729456618,Lunch Ride,79588.5,11082,14430,202.0,Ride,Ride,NaN,2025-09-07T09:45:26Z,2025-09-07T11:45:26Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,44,13,0,1,0,False,False,False,False,followers_only,False,"[51.108316, 17.123345]","[51.107901, 17.123794]",7.182,11.18,NaN,183.2,NaN,NaN,False,2029.7,True,129.0,148.0,False,True,158.4,115.4,1.680192e+10,16801924720,garmin_ping_477733386080,False,20,0,False,53.0,Nogi nie współpracowały po wczorajszym longu🪦,1388.0,None,None,False,Garmin Edge 840,246e2c4b121e976191c2c4d98a055e939da834c6,"[heartrate, power]",a15729456618,b12572672
1,15716821076,24km Race Practice Long Run🔩,24120.3,8004,8085,56.0,Run,Run,2.0,2025-09-06T10:41:12Z,2025-09-06T12:41:12Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,17,13,2,2,0,False,False,False,False,everyone,False,"[51.107164, 17.123723]","[51.106689, 17.123415]",3.014,4.28,84.8,369.3,581.0,375.0,True,2956.2,True,154.9,173.0,False,True,123.2,111.4,1.678855e+10,16788554675,garmin_ping_477379370380,False,10,0,False,165.0,24km Race Practice Long Run with Runna ✅\n\nDo...,1857.0,None,None,False,Garmin Forerunner 970,03edfc9cb2149366844c599b974197a1867550de,"[heartrate, pace, power]",a15716821076,g23642256
2,15708639235,Evening Ride,16823.7,3683,6122,47.0,Ride,Ride,NaN,2025-09-05T16:31:17Z,2025-09-05T18:31:17Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,5,10,0,4,0,False,False,False,False,followers_only,False,"[51.107755, 17.123295]","[51.107903, 17.12546]",4.568,12.62,NaN,95.9,NaN,NaN,False,353.1,True,101.0,145.0,False,True,126.2,116.2,1.677985e+10,16779848219,garmin_ping_477117769680,False,0,1,False,9.0,Coffee ride bez kawy🗿,320.0,None,None,False,Garmin Edge 840,18e8ba5d1d14645abe0ae5312e4a54cbc971c4ce,"[heartrate, power]",a15708639235,b12572672
3,15705468575,Afternoon Weight Training,0.0,3713,3713,0.0,Workout,WeightTraining,NaN,2025-09-05T12:00:34Z,2025-09-05T14:00:34Z,(GMT+02:00) Africa/Blantyre,7200.0,None,None,None,0,8,0,1,0,True,False,False,False,followers_only,False,[],[],0.000,0.00,NaN,NaN,NaN,NaN,None,NaN,True,99.6,142.0,False,True,0.0,0.0,1.677651e+10,16776511359,garmin_ping_477033364177,False,0,0,False,8.0,Reska8️⃣5️⃣,306.0,None,None,False,Garmin Forerunner 970,40d144b824a5b7046d70fc8f1b6889985fc50549,[heartrate],a15705468575,None
4,15705659558,Afternoon Ride,13045.3,1871,6563,44.0,Ride,Ride,10.0,2025-09-05T11:32:00Z,2025-09-05T13:32:00Z,(GMT+01:00) Europe/Warsaw,7200.0,None,None,None,9,12,0,1,0,False,True,False,False,followers_only,False,"[51.1085, 17.123504]","[51.107656, 17.125015]",6.972,10.50,NaN,181.2,NaN,NaN,False,339.1,True,134.1,152.0,False,True,129.0,115.4,1.677671e+10,16776713564,garmin_ping_477039270074,False,4,0,False,13.0,Reska dojazd🫡,318.0,None,None,False,Garmin Edge 840,964e3d803ba39b9bd7fdf466aa896620d5af75b5,"[heartrate, power]",a15705659558,b12572672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,14731708283,„Long” Run🙂‍↕️,11021.1,4111,4367,71.0,Run,Run,2.0,2025-06-08T08:31:47Z,2025-06-08T10:31:47Z,(GMT+01:00) Europe/Warsaw,7200.0,N

In [61]:
activities_df['moving_time_td'] = pd.to_timedelta(activities_df["moving_time"], unit="s")
activities_df['elapsed_time_td'] = pd.to_timedelta(activities_df["elapsed_time"], unit="s")

activities_df["start_date_dt"] = pd.to_datetime(activities_df["start_date"], utc=True)
activities_df["timezone_name"] = activities_df["timezone"].str.extract(r'\)\s*(.*)')
activities_df["start_date_local_dt"] = activities_df.apply(
    lambda row: row["start_date_dt"].tz_convert(row["timezone_name"]),
    axis=1
)

activities_df[["start_lat", "start_lng"]]  = pd.DataFrame(activities_df["start_latlng"].tolist(), index=activities_df.index)
activities_df[["end_lat", "end_lng"]] = pd.DataFrame(activities_df["end_latlng"].tolist(), index=activities_df.index)

activities_df['avg_pace_str'] = activities_df.apply(
  lambda row: speed_to_pace_str(row['average_speed']) if row['type'] == 'Run' else np.nan, axis=1
)

activities_df['avg_pace_float'] = activities_df.apply(
  lambda row: speed_to_pace_float(row['average_speed']) if row['type'] == 'Run' else np.nan, axis=1
)

activities_df['max_pace_str'] = activities_df.apply(
  lambda row: speed_to_pace_str(row['max_speed']) if row['type'] == 'Run' else np.nan, axis=1
)

activities_df['max_pace_float'] = activities_df.apply(
  lambda row: speed_to_pace_float(row['max_speed']) if row['type'] == 'Run' else np.nan, axis=1
)

In [62]:
activities_cols_clean = [
    'id',
    'name',
    'start_date_dt',
    'start_date_local_dt',
    'timezone_name',
    'distance',
    'moving_time',
    'moving_time_td',
    'elapsed_time',
    'elapsed_time_td',
    'total_elevation_gain',
    'elev_low',
    'elev_high',
    'type',
    'sport_type',
    'workout_type',
    'achievement_count',
    'kudos_count',
    'comment_count',
    'athlete_count',
    'photo_count',
    'trainer',
    'commute',
    'manual',
    'visibility',
    'average_speed',
    'avg_pace_str',
    'avg_pace_float',
    'max_speed',
    'max_pace_str',
    'max_pace_float',
    'average_cadence',
    'average_watts',
    'max_watts',
    'weighted_average_watts',
    # 'device_watts',
    # 'kilojoules',
    'has_heartrate',
    'average_heartrate',
    'max_heartrate',
    'pr_count',
    'total_photo_count',
    'suffer_score',
    'description',
    'calories',
    'device_name',
    'map_id',
    'gear_id'
]
activities_df = activities_df[activities_cols_clean]

In [63]:
activities_df_dtype_map = {
    "id": Integer,
    "name": String,
    "start_date_dt": DateTime(timezone=True),
    "start_date_local_dt":  DateTime(timezone=True),
    "timezone_name": String,
    "distance": Float,
    "moving_time": Integer,
    "moving_time_td": Interval,
    "elapsed_time": Integer,
    "elapsed_time_td": Interval,
    "total_elevation_gain": Float,
    "elev_low": Float,
    "elev_high": Float,
    "type": String,
    "sport_type": String,
    "workout_type": Float,
    "achievement_count": Integer,
    "kudos_count": Integer,
    "comment_count": Integer,
    "athlete_count": Integer,
    "photo_count": Integer,
    "trainer": Boolean,
    "commute": Boolean,
    "manual": Boolean,
    "visibility": String,
    "average_speed": Float,
    "avg_pace_str": String,
    "avg_pace_float": Float,
    "max_speed": Float,
    "max_pace_str": String,
    "max_pace_float": Float,
    "average_cadence": Float,
    "average_watts": Float,
    "max_watts": Float,
    "weighted_average_watts": Float,
    # "device_watts": String,
    # "kilojoules": Float,
    "has_heartrate": Boolean,
    "average_heartrate": Float,
    "max_heartrate": Float,
    "pr_count": Integer,
    "total_photo_count": Integer,
    "suffer_score": Float,
    "description": Text,
    "calories": Float,
    "device_name": String,
    "map_id": String,
    "gear_id": String,
}


In [64]:
activities_df

,id,name,start_date_dt,start_date_local_dt,timezone_name,distance,moving_time,moving_time_td,elapsed_time,elapsed_time_td,total_elevation_gain,elev_low,elev_high,type,sport_type,workout_type,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,visibility,average_speed,avg_pace_str,avg_pace_float,max_speed,max_pace_str,max_pace_float,average_cadence,average_watts,max_watts,weighted_average_watts,has_heartrate,average_heartrate,max_heartrate,pr_count,total_photo_count,suffer_score,description,calories,device_name,map_id,gear_id
0,15729456618,Lunch Ride,2025-09-07 09:45:26+00:00,2025-09-07 11:45:26+02:00,Europe/Warsaw,79588.5,11082,0 days 03:04:42,14430,0 days 04:00:30,202.0,115.4,158.4,Ride,Ride,NaN,44,13,0,1,0,False,False,False,followers_only,7.182,NaN,NaN,11.18,NaN,NaN,NaN,183.2,NaN,NaN,True,129.0,148.0,20,0,53.0,Nogi nie współpracowały po wczorajszym longu🪦,1388.0,Garmin Edge 840,a15729456618,b12572672
1,15716821076,24km Race Practice Long Run🔩,2025-09-06 10:41:12+00:00,2025-09-06 12:41:12+02:00,Europe/Warsaw,24120.3,8004,0 days 02:13:24,8085,0 days 02:14:45,56.0,111.4,123.2,Run,Run,2.0,17,13,2,2,0,False,False,False,everyone,3.014,5:32,5.529750,4.28,3:54,3.894081,84.8,369.3,581.0,375.0,True,154.9,173.0,10,0,165.0,24km Race Practice Long Run with Runna ✅\n\nDo...,1857.0,Garmin Forerunner 970,a15716821076,g23642256
2,15708639235,Evening Ride,2025-09-05 16:31:17+00:00,2025-09-05 18:31:17+02:00,Europe/Warsaw,16823.7,3683,0 days 01:01:23,6122,0 days 01:42:02,47.0,116.2,126.2,Ride,Ride,NaN,5,10,0,4,0,False,False,False,followers_only,4.568,NaN,NaN,12.62,NaN,NaN,NaN,95.9,NaN,NaN,True,101.0,145.0,0,1,9.0,Coffee ride bez kawy🗿,320.0,Garmin Edge 840,a15708639235,b12572672
3,15705468575,Afternoon Weight Training,2025-09-05 12:00:34+00:00,2025-09-05 14:00:34+02:00,Africa/Blantyre,0.0,3713,0 days 01:01:53,3713,0 days 01:01:53,0.0,0.0,0.0,Workout,WeightTraining,NaN,0,8,0,1,0,True,False,False,followers_only,0.000,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,True,99.6,142.0,0,0,8.0,Reska8️⃣5️⃣,306.0,Garmin Forerunner 970,a15705468575,None
4,15705659558,Afternoon Ride,2025-09-05 11:32:00+00:00,2025-09-05 13:32:00+02:00,Europe/Warsaw,13045.3,1871,0 days 00:31:11,6563,0 days 01:49:23,44.0,115.4,129.0,Ride,Ride,10.0,9,12,0,1,0,False,True,False,followers_only,6.972,NaN,NaN,10.50,NaN,NaN,NaN,181.2,NaN,NaN,True,134.1,152.0,4,0,13.0,Reska dojazd🫡,318.0,Garmin Edge 840,a15705659558,b12572672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,14731708283,„Long” Run🙂‍↕️,2025-06-08 08:31:47+00:00,2025-06-08 10:31:47+02:00,Europe/Warsaw,11021.1,4111,0 days 01:08:31,4367,0 days 01:12:47,71.0,23.8,77.8,Run,Run,2.0,0,14,0,1,0,False,False,False,everyone,2.681,6:13,6.216586,3.58,4:39,4.655493,83.5,343.8,463.0,335.0,True,145.8,158.0,0,0,54.0,11km Long Run with Runna ✅\n\nLuźno po dzielni...,857.0,Garmin Forerunner 970,a14731708283,g20426652
96,14722686686,XXXI Bieg Żakowski🔥,2025-06-07 11:01:15+00:00,2025-06-07 13:01:15+02:00,Europe/Warsaw,5047.6,2057,0 days 00:34:17,2062,0 days 00:34:22,41.0,32.6,69.8,Run,Run,1.0,0,19,0,4,0,False,False,False,everyone,2.448,6:48,6.808279,3.48,4:47,4.789272,78.8,309.2,486.0,309.0,True,146.9,164.0,0,1,29.0,Karo poprowadzona na nowy PR🏆,416.0,Garmin Forerunner 970,a14722686686,g23642256
97,14707040076,Fast 8-4-2s🚀,2025-06-05 17:15:13+00:00,2025-06-05 19:15:13+02:00,Europe/Warsaw,8732.2,2892,0 days 00:48:12,2892,0 days 00:48:12,3.0,115.0,119.2,Run,Run,3.0,7,13,2,3,0,False,False,False,everyone,3.019,5:31,5.520592,4.98,3:21,3.346720,78.0,348.0,583.0,381.0,True,157.8,179.0,3,0,73.0,Fast 8-4-2s with Runna ✅\n\nNogi w końcu dobrz...,666.0,Garmin Forerunner 970,a14707040076,g23642256
98,14694691688,Afternoon Weight Training,2025-06-04 13:58:52+00:00,2025-06-04 15:58:52+02:00,Africa/Blantyre,0.0,3763,0 days 01:02:43,3763,0 days 01:02:43,0.0,0.0,0.0,WeightTraining,Weight